In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
path = "D:/data cleaning/Chrome Driver/chromedriver-win64/chromedriver.exe"
s = Service(path)
driver = webdriver.Chrome(service=s)
time.sleep(3)
driver.get("https://www.google.com/maps")
time.sleep(3)
box = driver.find_element(By.XPATH, """//*[@id="searchboxinput"]""")
box.send_keys("banquets in ghaziabad")
box.send_keys(Keys.RETURN)
time.sleep(2)

In [3]:
scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')


In [4]:
last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

while True:
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scrollable_div)
    time.sleep(2)
    new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    if new_height == last_height:
        break
    last_height = new_height
time.sleep(2)

In [5]:
soup = BeautifulSoup(driver.page_source,'html.parser')

titles = []
ratings = []
links = []

for item in soup.find_all('div', class_ = 'Nv2PK THOPZb CpccDe'):
    a_tag = item.find('a', class_='hfpxzc')
    try:
        titles.append(a_tag.get('aria-label'))
        links.append(a_tag.get('href'))
    except:
        titles.append(np.nan)
        links.append(np.nan)
    
    rating_span = item.find('span', class_='MW4etd')
    if rating_span:
        ratings.append(rating_span.text)
    else:
        ratings.append(np.nan)
        
print(len(titles) == len(ratings) == len(links))
print(len(titles))

True
120


In [7]:
data = []
for i in range(len(titles)):
    data.append([titles[i],ratings[i],links[i]])
print(len(data))

120


In [9]:
df = pd.DataFrame(data, columns = ['Name','Rating','link'])
df

,Name,Rating,link
0,Rosellia Banquets & Suites,4.5,https://www.google.com/maps/place/Rosellia+Ban...
1,Orchid Farms | Best Banquet Hall in Indirapura...,4.5,https://www.google.com/maps/place/Orchid+Farms...
2,Wow Palace Banquet | Best Banquet in indirapur...,4.0,https://www.google.com/maps/place/Wow+Palace+B...
3,The Grand Piazza,4.4,https://www.google.com/maps/place/The+Grand+Pi...
4,The Royal Palace,3.7,https://www.google.com/maps/place/The+Royal+Pa...
...,...,...,...
115,Kalash A.C. Banquet and Party Lawn,4.3,https://www.google.com/maps/place/Kalash+A.C.+...
116,Royal Grand Banquets,4.3,https://www.google.com/maps/place/Royal+Grand+...
117,Shree Nath Jee Hotel & Banquet,3.9,https://www.google.com/maps/place/Shree+Nath+J...
118,Royal Dream The Banquet,4.3,https://www.google.com/maps/place/Royal+Dream+...


In [10]:
df.to_csv("banquets.csv")